# Dynamic sensitivity analysis
ok so we have these components
* initial conditions (in `initial_conditions.txt`)
* x<sub>i</sub>(t) for all i in [1, 24] (in `x(t).csv`)
* sensitivity matrix (sbar) as a function of k<sub>j</sub> and x<sub>i</sub> (in `sensitivity_matrix` pickle file)
* k<sub>j</sub> for all j [1, 64] (in `reaction_coefficients.txt`)

Using these components, we can calculate equations (21) and (22)

### load all these bad bois

In [16]:
# get x(t) for all t
xt = {'x{}'.format(i+1): [] for i in range(24)}
with open('./x(t).csv', 'r') as o:
    header = False
    for line in o:
        if not header:
            header = True
            continue
        vals = line.split(',')[1:]
        numvals = [float(x) for x in vals]
        for i in range(len(numvals)):
            xt['x{}'.format(i+1)].append(numvals[i])
        
N = len(xt['x1'])

# get the sensitivity matrix
import pickle
sbar = pickle.load(open('./sensitivity_matrix', 'rb'))

# get all of the ks
ks = {}
with open('./reaction_coefficients.txt', 'r') as o:
    for line in o:
        k = line.split('=')[0]
        ks[k] = sp.sympify(line.split('=')[1].replace('\n', '').strip()).evalf()

## Dynamic sensitivity analysis with multiple variables
This is equation 22 of the paper
![Equation 22](./equation22.png)

In [24]:
import sympy as sp
#calc sbar 
def s_bar(i, j, k):
    # get the row of xt for the k
    xs = [(str(key), xt[key][k]) for key in xt.keys()]
    # get the i j entry of sbar
    s = sbar[i][j]
    # substitute ks and xs and return
    ss = sp.sympify(s)
    
    print(type(ss))
    print(ss)
    for key in ks:
        print('subed {} in for {}'.format(ks[key], key))
        ss.sub(key, str(ks[key]))
    print(ss)
    for x in xs:
        ss.sub(x[0], x[1])
    print(ss)
    a = ss.evalf()
    print(a)
    return a

In [25]:
s_bar(1, 2, 3)

<class 'sympy.core.mul.Mul'>
-k3*x4
subed 0.500000000000000 in for k1


AttributeError: 'Mul' object has no attribute 'sub'